In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import os
import socket
import uuid
import numpy as np
from datetime import datetime
from scipy.stats import norm

# === Settings ===
OUTPUT_DIR = r"C:\Users\deepe\OneDrive\Documents\Qnatitative Trading Analysis (Chris)\Source"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Utility ===
def safe_filename(name):
    return name.lower().replace(' ', '_').replace('/', '_')

def unique_filename(base, ext):
    uid = uuid.uuid4().hex[:6]
    return f"{safe_filename(base)}_{uid}.{ext}"

# === Tor Proxy Detection ===
def is_tor_port_open(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.settimeout(1)
        try:
            s.connect(("127.0.0.1", port))
            return True
        except:
            return False

def get_working_tor_proxy():
    for port in [9050, 9150]:
        if is_tor_port_open(port):
            print(f"✅ Found Tor running on port {port}")
            return {
                'http': f'socks5h://127.0.0.1:{port}',
                'https': f'socks5h://127.0.0.1:{port}'
            }
    raise ConnectionError("❌ Tor is not running on ports 9050 or 9150. Please start Tor first.")

# === Fetching Data ===
def fetch_coinbase_data_through_tor(product_id="BTC-USD", granularity=86400, limit=300):
    proxies = get_working_tor_proxy()
    url = f"https://api.exchange.coinbase.com/products/{product_id}/candles?granularity={granularity}&limit={limit}"
    print("🧅 Fetching data via Tor from Coinbase...")
    try:
        response = requests.get(url, proxies=proxies, timeout=10)
        response.raise_for_status()
        print("✅ Data fetched successfully!")
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Data fetch failed: {e}")
        return None

# === Cleaning and Normalization ===
def clean_coinbase_data(raw_data):
    df = pd.DataFrame(raw_data, columns=["timestamp", "low", "high", "open", "close", "volume"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df = df.sort_values("timestamp").reset_index(drop=True)
    df = df.astype({"low": float, "high": float, "open": float, "close": float, "volume": float})
    df["log_close"] = np.log(df["close"])
    return df

# === Strategy ===
def sma_crossover_strategy(df, short_window=5, long_window=20, initial_capital=100000, trade_size=10000):
    df["SMA_Short"] = df["close"].rolling(window=short_window).mean()
    df["SMA_Long"] = df["close"].rolling(window=long_window).mean()
    df["Buy or Sell"] = ""
    df["Cash"] = float(initial_capital)
    df["Position"] = 0.0
    df["Portfolio Value"] = float(initial_capital)

    position = 0.0
    capital = initial_capital
    max_portfolio = initial_capital
    drawdowns = []
    trades = []

    for i in range(1, len(df)):
        prev_short = df["SMA_Short"].iloc[i - 1]
        prev_long = df["SMA_Long"].iloc[i - 1]
        curr_short = df["SMA_Short"].iloc[i]
        curr_long = df["SMA_Long"].iloc[i]
        price = df["close"].iloc[i]

        if pd.notna(prev_short) and pd.notna(prev_long):
            if curr_short > curr_long and prev_short <= prev_long and capital >= trade_size:
                position += trade_size / price
                capital -= trade_size
                df.at[i, "Buy or Sell"] = "Buy"
                trades.append(("Buy", price))
            elif curr_short < curr_long and prev_short >= prev_long and position > 0:
                capital += position * price
                df.at[i, "Buy or Sell"] = "Sell"
                trades.append(("Sell", price))
                position = 0.0

        portfolio_value = capital + position * price
        max_portfolio = max(max_portfolio, portfolio_value)
        drawdown = (max_portfolio - portfolio_value) / max_portfolio
        drawdowns.append(drawdown)

        df.at[i, "Cash"] = float(capital)
        df.at[i, "Position"] = float(position)
        df.at[i, "Portfolio Value"] = float(portfolio_value)

    wins = losses = 0
    returns = []
    gains = []
    losses_list = []

    for j in range(1, len(trades), 2):
        if j + 1 < len(trades) and trades[j][0] == "Sell" and trades[j - 1][0] == "Buy":
            pnl = trades[j][1] - trades[j - 1][1]
            ret = pnl / trades[j - 1][1]
            returns.append(ret)
            if pnl > 0:
                wins += 1
                gains.append(pnl)
            else:
                losses += 1
                losses_list.append(abs(pnl))

    win_rate = wins / (wins + losses) if (wins + losses) > 0 else 0
    loss_rate = losses / (wins + losses) if (wins + losses) > 0 else 0
    max_drawdown = max(drawdowns) if drawdowns else 0
    sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252) if returns else 0
    total_return = (df["Portfolio Value"].iloc[-1] - initial_capital) / initial_capital
    avg_gain = np.mean(gains) if gains else 0
    avg_loss = np.mean(losses_list) if losses_list else 0
    pl_ratio = avg_gain / avg_loss if avg_loss != 0 else float('inf')

    print("\n📌 Strategy Stats:")
    print(f"- Number of trades: {(wins + losses)}")
    print(f"- Final portfolio value: ${df['Portfolio Value'].iloc[-1]:,.2f}")
    print(f"- Total return: {total_return:.2%}")
    print(f"- P/L ratio: {pl_ratio:.2f}")

    return df, win_rate, loss_rate, max_drawdown, sharpe_ratio

# === Save CSV ===
def save_to_csv(df, base_name):
    filename = unique_filename(base_name, "csv")
    path = os.path.join(OUTPUT_DIR, filename)
    df.to_csv(path, index=False)
    print(f"📁 Data saved to: {path}")

# === Plotting ===
def plot_sma_signals(df, title):
    plt.figure(figsize=(14, 6))
    plt.plot(df["timestamp"], df["close"], label="Close Price", alpha=0.5)
    plt.plot(df["timestamp"], df["SMA_Short"], label="SMA Short", color="green")
    plt.plot(df["timestamp"], df["SMA_Long"], label="SMA Long", color="red")

    buys = df[df["Buy or Sell"] == "Buy"]
    sells = df[df["Buy or Sell"] == "Sell"]
    plt.scatter(buys["timestamp"], buys["close"], marker="^", color="blue", label="Buy")
    plt.scatter(sells["timestamp"], sells["close"], marker="v", color="black", label="Sell")

    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.grid()

    filename = unique_filename(title, "png")
    path = os.path.join(OUTPUT_DIR, filename)
    plt.savefig(path)
    print(f"📈 Chart saved to: {path}")
    plt.close()

# === Main ===
if __name__ == "__main__":
    raw_coinbase = fetch_coinbase_data_through_tor()
    if raw_coinbase:
        df_cb = clean_coinbase_data(raw_coinbase)
        df_cb, win_cb, loss_cb, max_dd_cb, sharpe_cb = sma_crossover_strategy(df_cb)
        print(f"\n📊 Coinbase - Win Rate: {win_cb:.2%}, Loss Rate: {loss_cb:.2%}, Max DD: {max_dd_cb:.2%}, Sharpe: {sharpe_cb:.2f}")
        save_to_csv(df_cb, "sma_coinbase")
        plot_sma_signals(df_cb, "Coinbase SMA Strategy")

    print("\n✅ Full backtest completed.")


✅ Found Tor running on port 9150
🧅 Fetching data via Tor from Coinbase...
✅ Data fetched successfully!

📌 Strategy Stats:
- Number of trades: 8
- Final portfolio value: $103,519.10
- Total return: 3.52%
- P/L ratio: 4.71

📊 Coinbase - Win Rate: 25.00%, Loss Rate: 75.00%, Max DD: 2.88%, Sharpe: 2.85
📁 Data saved to: C:\Users\deepe\OneDrive\Documents\Qnatitative Trading Analysis (Chris)\Source\sma_coinbase_0fb4bf.csv
📈 Chart saved to: C:\Users\deepe\OneDrive\Documents\Qnatitative Trading Analysis (Chris)\Source\coinbase_sma_strategy_3ae1a1.png

✅ Full backtest completed.
